2018/9/27 Updates

Changes
1. More columns convertions to proper datatypes
2. New dataframe created for future manipulations
3. Entry and exit tracking numbers combined for each turnstile reporting
4. Hours elapsed since previous reporting tracked
5. Passenger per hour per turntiles tracked


Next steps:
2. Rate for each station by hour (groupby station, day, hour)


ID explained 
Station then Unit(think enterance maybe) then CA(booth) then SCP(turnstyle)

In [1]:
import pandas as pd
import re

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
#load April - July, 2017 & 2018 MTA Turntile data
#time intensive block
df = pd.read_csv('/Users/GabeKlick/Downloads/turnstile_recordings_summer_2017_2018_clean.csv', index_col = 'Unnamed: 0')
df.head()

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,00:00:00,REGULAR,6157740,2085315
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,REGULAR,6157777,2085319
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,REGULAR,6157810,2085353
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,REGULAR,6157963,2085453
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,REGULAR,6158212,2085529


In [4]:
#rename index column to be used later for dateframes merges
df.rename(index = {'Unnamed: 0': 'Index'}, inplace = True)
df.head()

,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,00:00:00,REGULAR,6157740,2085315
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,REGULAR,6157777,2085319
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,REGULAR,6157810,2085353
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,REGULAR,6157963,2085453
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,REGULAR,6158212,2085529


In [5]:
#exit column has a few misplaced chars preventing the column from being int values
#this block removes the chars and converts columns to int64

#apply regex statement to ensure only numbers in the column
def only_num(col):
    return re.findall('(\d)', col)
df.exits = df.exits.apply(lambda x: only_num(str(x)))

#the regex statment returns a list a numbers for each value
#join list values to single string items of numbers
df.exits = df.exits.apply(lambda x: ''.join(x))

#convert str to int64
df.exits = df.exits.apply(lambda x: int(x))

df.head()

,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,00:00:00,REGULAR,6157740,2085315
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,REGULAR,6157777,2085319
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,REGULAR,6157810,2085353
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,REGULAR,6157963,2085453
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,REGULAR,6158212,2085529


In [6]:
#combine and convert date,time column from str to datetime
#time intensive block, run then take 5

#df.date = pd.to_datetime(df.date_time, infer_datetime_format= True)
df.time = pd.to_datetime(df.date + ' ' + df.time)
df.date = pd.to_datetime(df.date)

#rename to showcase the new time feature in the column
df.rename({'time': 'date_time'}, axis = 'columns', inplace = True)

#removes the now redundant time column
#df.drop(columns = ['time'], inplace = True)

df.head()

,c_a,unit,scp,station,linename,division,date,date_time,desc,entries,exits
0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,2017-04-29 00:00:00,REGULAR,6157740,2085315
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,2017-04-29 04:00:00,REGULAR,6157777,2085319
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,2017-04-29 08:00:00,REGULAR,6157810,2085353
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,2017-04-29 12:00:00,REGULAR,6157963,2085453
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29,2017-04-29 16:00:00,REGULAR,6158212,2085529


In [7]:
#create pickle for quicker processing time
df.to_pickle('date_formatted_df.pkl')

In [8]:
df = pd.read_pickle('date_formatted_df.pkl')

In [9]:
#remove dates before May and after August
df = df[(df.date_time.dt.month != 4) & (df.date_time.dt.month != 8)].reset_index()
df.head()

,index,c_a,unit,scp,station,linename,division,date,date_time,desc,entries,exits
0,12,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01,2017-05-01 00:00:00,REGULAR,6159107,2085842
1,13,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01,2017-05-01 04:00:00,REGULAR,6159116,2085849
2,14,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01,2017-05-01 08:00:00,REGULAR,6159166,2085976
3,15,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01,2017-05-01 12:00:00,REGULAR,6159331,2086218
4,16,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01,2017-05-01 16:00:00,REGULAR,6159595,2086299


In [10]:
#replace typo
df[df['c_a'] == ' "A002']['c_a'] = 'A002'

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
#grouping by turnstile id (station, unit, c_a, scp), find the difference between consecutive data points
#for each individual turnstile. This method is applied to both the exit, entry, and date_time for each observation
difference_of_entries = df.groupby(['station', 'unit', 'c_a', 'scp'])['entries'].apply(lambda x: x- x.shift(1))
difference_of_exits = df.groupby(['station', 'unit', 'c_a', 'scp'])['exits'].apply(lambda x: x- x.shift(1))
time_elapsed = df.groupby(['station', 'unit', 'c_a', 'scp'])['date_time'].apply(lambda x: x- x.shift(1))

#these 3 series are ardded to a temp data frame that will be merged to the main dataframe
temp_df = pd.DataFrame({'Index': df.index, 'num_entries': difference_of_entries, 'num_exits': difference_of_exits, 'time_elapsed': time_elapsed})

In [12]:
#time intensive, run then take 5
#merge the main and temp df on the index column
df = pd.concat([df, temp_df], axis = 1, join = 'outer')
df.head()

,index,c_a,unit,scp,station,linename,division,date,date_time,desc,entries,exits,Index,num_entries,num_exits,time_elapsed
0,12,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01,2017-05-01 00:00:00,REGULAR,6159107,2085842,0,nan,nan,NaT
1,13,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01,2017-05-01 04:00:00,REGULAR,6159116,2085849,1,9.000,7.000,04:00:00
2,14,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01,2017-05-01 08:00:00,REGULAR,6159166,2085976,2,50.000,127.000,04:00:00
3,15,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01,2017-05-01 12:00:00,REGULAR,6159331,2086218,3,165.000,242.000,04:00:00
4,16,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-05-01,2017-05-01 16:00:00,REGULAR,6159595,2086299,4,264.000,81.000,04:00:00


In [13]:
#create a new data frame for future manipulation
station_df = df[['station', 'date', 'date_time','num_entries', 'num_exits', 'time_elapsed']]
station_df.head()

,station,date,date_time,num_entries,num_exits,time_elapsed
0,59 ST,2017-05-01,2017-05-01 00:00:00,nan,nan,NaT
1,59 ST,2017-05-01,2017-05-01 04:00:00,9.000,7.000,04:00:00
2,59 ST,2017-05-01,2017-05-01 08:00:00,50.000,127.000,04:00:00
3,59 ST,2017-05-01,2017-05-01 12:00:00,165.000,242.000,04:00:00
4,59 ST,2017-05-01,2017-05-01 16:00:00,264.000,81.000,04:00:00


In [14]:
#combine total passengers for each turnstile reporting
station_df['total_passengers'] = df.num_entries + df.num_exits
station_df.head()

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,station,date,date_time,num_entries,num_exits,time_elapsed,total_passengers
0,59 ST,2017-05-01,2017-05-01 00:00:00,nan,nan,NaT,nan
1,59 ST,2017-05-01,2017-05-01 04:00:00,9.000,7.000,04:00:00,16.000
2,59 ST,2017-05-01,2017-05-01 08:00:00,50.000,127.000,04:00:00,177.000
3,59 ST,2017-05-01,2017-05-01 12:00:00,165.000,242.000,04:00:00,407.000
4,59 ST,2017-05-01,2017-05-01 16:00:00,264.000,81.000,04:00:00,345.000


In [15]:
#create column tracking total hours passed for each turnstile reporting
station_df['hours_elapsed'] = ((station_df.time_elapsed.dt.days * 86400) + (station_df.time_elapsed.dt.seconds))/3600
station_df.head()

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,station,date,date_time,num_entries,num_exits,time_elapsed,total_passengers,hours_elapsed
0,59 ST,2017-05-01,2017-05-01 00:00:00,nan,nan,NaT,nan,nan
1,59 ST,2017-05-01,2017-05-01 04:00:00,9.000,7.000,04:00:00,16.000,4.000
2,59 ST,2017-05-01,2017-05-01 08:00:00,50.000,127.000,04:00:00,177.000,4.000
3,59 ST,2017-05-01,2017-05-01 12:00:00,165.000,242.000,04:00:00,407.000,4.000
4,59 ST,2017-05-01,2017-05-01 16:00:00,264.000,81.000,04:00:00,345.000,4.000


In [16]:
#for each turnstile reporting, finds the rate of passengers per hour
station_df['passengers_per_hour'] = station_df.total_passengers/station_df.hours_elapsed
station_df.head()

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,station,date,date_time,num_entries,num_exits,time_elapsed,total_passengers,hours_elapsed,passengers_per_hour
0,59 ST,2017-05-01,2017-05-01 00:00:00,nan,nan,NaT,nan,nan,nan
1,59 ST,2017-05-01,2017-05-01 04:00:00,9.000,7.000,04:00:00,16.000,4.000,4.000
2,59 ST,2017-05-01,2017-05-01 08:00:00,50.000,127.000,04:00:00,177.000,4.000,44.250
3,59 ST,2017-05-01,2017-05-01 12:00:00,165.000,242.000,04:00:00,407.000,4.000,101.750
4,59 ST,2017-05-01,2017-05-01 16:00:00,264.000,81.000,04:00:00,345.000,4.000,86.250


In [17]:
station_df.to_csv('station_df.csv')

In [18]:
station_df = pd.read_csv('station_df.csv')

In [19]:
aggregations = {
    'total_passengers': 'sum',
    'hours_elapsed': 'sum'
}

In [20]:
station_df.groupby(('station','date')).agg(aggregations).reset_index()
station_df.rename({'hours_elapsed': 'total_hours'}, axis = 1, inplace = True)
station_df.head()

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


,Unnamed: 0,station,date,date_time,num_entries,num_exits,time_elapsed,total_passengers,total_hours,passengers_per_hour
0,0,59 ST,2017-05-01 00:00:00,2017-05-01 00:00:00,nan,nan,NaN,nan,nan,nan
1,1,59 ST,2017-05-01 00:00:00,2017-05-01 04:00:00,9.000,7.000,0 days 04:00:00.000000000,16.000,4.000,4.000
2,2,59 ST,2017-05-01 00:00:00,2017-05-01 08:00:00,50.000,127.000,0 days 04:00:00.000000000,177.000,4.000,44.250
3,3,59 ST,2017-05-01 00:00:00,2017-05-01 12:00:00,165.000,242.000,0 days 04:00:00.000000000,407.000,4.000,101.750
4,4,59 ST,2017-05-01 00:00:00,2017-05-01 16:00:00,264.000,81.000,0 days 04:00:00.000000000,345.000,4.000,86.250


Unused Code Below

#Entry and Exits data recorded as a 'snapshot' every 4 hours
#Adding columns tracking pasenger entery and exit evert 4 hours
df = df.assign(entry_count = df['entries'] - df['entries'].shift(1))
df = df.assign(exit_count = df['exits'] - df['exits'].shift(1))

This block can be used for the challenge but not neccessary for our goals for the project
returns values in the formate requested by the challages 

#This blocks combines the date and time columns so they may later be converted to a datetime type

#combines columns, separated by a space, into the pre-exisiting column 'date'
df.date = df.date + ' '+ df.time

#rename to showcase the new time feature in the column
df.rename({'date': 'date_time'}, axis = 'columns', inplace = True)

#removes the now redundant time column
df.drop(columns = ['time'], inplace= True)

#converting date_time from string to datetime
df.date_time = df.date_time.apply(lambda x: dateutil.parser.parse(x))


#time intensive. Run, then take 5
# df.date = df.date.apply(lambda x: dateutil.parser.parse(x))

Again, may be used for the challenge, but not necessary for the project

#create a new column that combines the date and time columns, separated by a single space
df['date_time'] = df.date + ' '+ df.time
#Warning: time intensive block, run then take 5
#converting date_time from string to datetime
df.date_time = df.date_time.apply(lambda x: dateutil.parser.parse(x))
df.head()
#currently returning datetime in same formate it is in: mm/dd/yyyy hh:mm:ss
#different than the challenge scope, but seems 

Incase any issues arrive with exit_entry function, this is the originial code

#This should be a function. See above
#time intensive. Run, then take 5

#Groups turnstiles by station, turnstyle id, and date
#within each group find the difference between the consecutive entry points 
#this difference is stored as a series
df.test = df.groupby(['station','scp','date']).entries.apply(lambda x: x - x.shift(1))

#create a temp df that uses the same index column as the main df

#merge the main and temp df on the index column
df = pd.merge(df, df_entry, how = 'outer', on = 'Index')

df.head(10)

This function is no longer being used

'''
Creates a new series that groups turnstyles by station, id, and date, 
and find the difference in consecutive data points
stores this difference as a series
crease a temp df using difference and the main df's index column
'''
def entry_exit_calc(column, new_col_name, index_name):
    #Groups turnstiles by station, turnstyle id, and date
    #within each group find the difference between the consecutive entry points 
    #this difference is stored as a series
    difference_of_entries = df.groupby(['station', 'unit', 'c_a', 'scp'])[column].apply(lambda x: x- x.shift(1))
    
    hours_elapsed = df.groupby(['station', 'unit', 'c_a', 'scp'])['date_time'].apply(lambda x: x- x.shift(1)) #make sure date_time is correct name
    #create a temp df that uses the same index column as the main df
    return pd.DataFrame({'Index': df[index_name], new_col_name: difference_of_tolls, hours_elapsed: hours_elapsed})
    